In [1]:
import json
import requests
import sys

# change_ids = ['I9bd5f80ada856b7db4b39dfb59b32bd825416c13']
change_ids = ['I32172ed44d74378c627918e19b9e1aaadb5c6d1d', 'I9b352ac98e2a961157f5bb36456bec3e35891270', 'I419c324634be8ee6884e02032bb53a42738305ac']

users = {}
changes = []
files = []
authors = []

def get_change_info(id):
  change = {}
  url = "https://codereview.qt-project.org/changes/?q={}+AND+branch:dev&o=DETAILED_ACCOUNTS&o=ALL_COMMITS&o=ALL_REVISIONS".format(id)
  try:
    res = requests.get(url)
    if res.status_code == 200:
      data = json.loads(res.text[6:-2])
      revisions = data["revisions"]
      commit_msg = ""
      parent = ""
      for revision in revisions.values():
        if revision["_number"] == 1:
          commit_msg = revision["commit"]["message"]
          parent = revision["commit"]["parents"][0]["commit"]
      change = {
        "change_id": data["change_id"],
        "project": data["project"],
        "branch": data["branch"],
        "subject": data["subject"],
        "status": data["status"],
        "created": data["created"],
        "updated": data["updated"],
        "submitted": data["submitted"] if "submitted" in data else None,
        "insertions": data["insertions"],
        "deletions": data["deletions"],
        "number": data["_number"],
        "author": data["owner"]["_account_id"],
        "commit_msg": commit_msg,
        "parent": parent
      }
      if data["owner"]["_account_id"] not in users:
        users[data["owner"]["_account_id"]] = {
          "name": data["owner"]["name"],
          "email": data["owner"]["email"],
          "username": data["owner"]["username"]
        }
        # change["submitter"] = data["submitter"]["_account_id"]
        # if data["submitter"]["_account_id"] in gerrit_users:
        #     gerrit_users[data["submitter"]["_account_id"]] = {
        #   "name": data["submitter"]["name"],
        #   "email": data["submitter"]["email"],
        #   "username": data["submitter"]["username"]
        # }
    else:
      print("Unexpected response code from REST server for change {}: {}".format(id, res.status_code), file=sys.stderr)
  except Exception as e:
    print("Unexpected exception for change {}: {}".format(id, e), file=sys.stderr)
  return change

def get_changed_files(id):
  files = []
  url = "https://codereview.qt-project.org/changes/qt%2Fqtbase~dev~{}/revisions/1/files".format(id)
  try:
    res = requests.get(url)
    if res.status_code == 200:
      data = json.loads(res.text[5:-1])
      for filename, file_info in data.items():
        file_diff = get_file_diff(id, filename.replace("/", "%2F"))
        files.append({
          "filename": filename,
          "status": file_info["status"] if "status" in file_info else None,
          "insertions": file_info["lines_inserted"] if "lines_inserted" in file_info else 0,
          "deletions": file_info["lines_deleted"] if "lines_deleted" in file_info else 0,
          "code_a": file_diff["code_a"],
          "code_b": file_diff["code_b"]
        })
    else:
      print("Unexpected response code from REST server for change {}: {}".format(id, res.status_code), file=sys.stderr)
  except Exception as e:
    print("Unexpected exception for change {}: {}".format(id, e), file=sys.stderr)
  return files

def get_file_diff(id, file):
  code_a = []
  code_b = []
  diff_header = []
  url = "https://codereview.qt-project.org/changes/qt%2Fqtbase~dev~{}/revisions/1/files/{}/diff?context=ALL&intraline&whitespace=IGNORE_NONE".format(id, file)
  try:
    res = requests.get(url)
    if res.status_code == 200:
      data = json.loads(res.text[5:-1])      
      for object in data["content"]:
        if "ab" in object:
          code_a = code_a + object['ab']
          code_b = code_b + object['ab']
        if "a" in object:
          code_a = code_a + object['a']
        if "b" in object:
          code_b = code_b + object['b']
    else:
      print("Unexpected response code from REST server for change {} file {}: {}".format(id, file, res.status_code), file=sys.stderr)
  except Exception as e:
    print("Unexpected exception for change {} file {}: {}".format(id, file, e), file=sys.stderr)
  return {
    "diff_header": "\n".join(diff_header),
    "code_a": "\n".join(code_a),
    "code_b": "\n".join(code_b)
  }

file_diff = {}
for id in change_ids:
  change = get_change_info(id)
  changes.append(change)
  change_files = get_changed_files(id)
  # file_diff = {}
  for file in change_files:
    file["change_id"] = id
    files.append(file)
    
for account_id, info in users.items():
  authors.append({
    "account_id": account_id,
    "name": info["name"],
    "email": info["email"],
    "username": info["username"]
  })

print(changes)
# print(files)
print(users)
print(len(changes))
print(len(files))
print(len(authors))
# print(file_diff['src/corelib/thread/qsemaphore.cpp']['code_a'])



[{'change_id': 'I32172ed44d74378c627918e19b9e1aaadb5c6d1d', 'project': 'qt/qtbase', 'branch': 'dev', 'subject': 'Fix race condition in futex-based QSemaphore', 'status': 'MERGED', 'created': '2022-04-12 14:26:24.000000000', 'updated': '2022-04-15 11:16:02.000000000', 'submitted': '2022-04-15 11:00:58.000000000', 'insertions': 6, 'deletions': 3, 'number': 405857, 'author': 1001132, 'commit_msg': 'Fix race condition in futex-based QSemaphore\n\nAdd one and reset the wakeAll bit atomically.\n\nPick-to: 6.3 6.2 5.15\nFixes: QTBUG-102484\nChange-Id: I32172ed44d74378c627918e19b9e1aaadb5c6d1d\n', 'parent': 'ca106e261e491c637681c22f537ba5d4eaf0513f'}, {'change_id': 'I9b352ac98e2a961157f5bb36456bec3e35891270', 'project': 'qt/qtbase', 'branch': 'dev', 'subject': 'wasm: use emscripten::val for specialHTMLTargets', 'status': 'MERGED', 'created': '2022-04-13 04:02:12.000000000', 'updated': '2022-04-15 11:06:32.000000000', 'submitted': '2022-04-15 11:06:32.000000000', 'insertions': 9, 'deletions': 7

In [2]:
import mysql.connector
from itertools import cycle

cnx = mysql.connector.connect(user='root', password='password',
                              host='localhost',
                              database='gherald')


cursor = cnx.cursor()
add_author = ("INSERT INTO author "
               "(account_id, email, name, username) "
               "VALUES (%s, %s, %s, %s)")
add_change = ("INSERT INTO change_detail "
               "(id, branch, commit_msg, created, deletions, insertions, number, parent, project, risk_level, status, subject, submitted, updated, author_account_id) "
               "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")
add_file = ("INSERT INTO file "
               "(id, codea, codeb, deletions, filename, insertions, status, change_id) "
               "VALUES (%s, %s, %s, %s, %s, %s, %s, %s)")

for author in authors:
  data_author = (author["account_id"], author["email"], author["name"], author["username"])
  cursor.execute(add_author, data_author)

risk_levels = cycle([1, 2, 3])
for change in changes:
  data_change = (change["change_id"], change["branch"], change["commit_msg"], change["created"], change["deletions"], change["insertions"], change["number"], change["parent"], change["project"], next(risk_levels), change["status"], change["subject"], change["submitted"], change["updated"], change["author"])
  cursor.execute(add_change, data_change)

file_id = 1
for file in files:
  data_file = (file_id, file["code_a"], file["code_b"], file["deletions"], file["filename"], file["insertions"], file["status"], file["change_id"])
  cursor.execute(add_file, data_file)
  file_id += 1

# val = [("Nikhil", "CSE", "98", "A", "18"),
#        ("Nisha", "CSE", "99", "A", "18"),
#        ("Rohan", "MAE", "43", "B", "20"),
#        ("Amit", "ECE", "24", "A", "21"),
#        ("Anil", "MAE", "45", "B", "20"),
#        ("Megha", "ECE", "55", "A", "22"),
#        ("Sita", "CSE", "95", "A", "19")]
   
# cursorObject.executemany(sql, val)

cnx.commit()

cursor.close()
cnx.close()